In [1]:
import numpy as np
import pandas as pd

In [5]:
# import course data from LP_Input.csv
# the index should be the list of courses + the V2 courses.
course_list = pd.DataFrame.from_csv("LP_Input.csv")

course_list = list(course_list.index)
course_list.append("missing")

hs_response = pd.DataFrame.from_csv("School form - High School form responses.csv")
ms_response = pd.DataFrame.from_csv("School form - Middle School form responses.csv")
hs_response = hs_response.fillna('missing')
ms_response = ms_response.fillna('missing')
hs_response = hs_response.drop( 'IIC Mathematics []', axis = 1)

change_dict = {'N/A': 'place holder', np.nan:'place holder',
             "HS English TBA": "Non-Western Writers",
             'Intermediate Algebra and Geometry': "Discovering Algebra",
             'Beginning Algebra and Geometry': "Discovering Geometry",
             'Advanced/In-Depth French': "Advanced French",
             'Social Studies (BE)': "Governance and Dissent",
             'MS PE':"MS PE "}

for val in change_dict:
    hs_response.replace(val, change_dict[val], inplace=True)
    ms_response.replace(val, change_dict[val], inplace=True)


In [7]:
hs_data = hs_response.iloc[:, 5:-10]
ms_data = ms_response.iloc[:, 5:-10]

In [8]:
# extract preference levels from the column names

temp_list = hs_response.columns[5:-10]
temp_list2 = ms_response.columns[5:-10]

# extracted choice numbers from the column names 
hs_choices = []
ms_choices = []

for item in temp_list:
    hs_choices.append(int(item[-2]))
    
for item in temp_list2:
    ms_choices.append(int(item[-2]))

In [9]:
result = pd.DataFrame(0,columns = course_list, index = range(hs_data.shape[0]+ms_data.shape[0]))

In [10]:
# helper function which returns the corresponding indices in course_list for each row of responses
def course_index(input_array):
    indices = []
    for item in input_array:
        indices.append(course_list.index(item))
    return np.array(indices)

In [11]:
for i in range(hs_data.shape[0]):
    # assign hs_choices values to the corresponding indices in the result data
    result.iloc[i,course_index(hs_data.iloc[i,:])] = hs_choices

    
ms_start_index = hs_data.shape[0]
for i in range(ms_data.shape[0]):
    # assign middle school choices. Row index is num_rows of hs_data + i
    # (so MS rows would be after HS rows in result)
    result.iloc[(ms_start_index + i),course_index(ms_data.iloc[i,:])] = ms_choices

In [15]:
result.iloc[3,course_index(hs_data.iloc[3,:])]

missing                       3
Non-Western Writers           1
I Know Some Things            2
Civil Rights Literature       3
missing                       3
missing                       3
missing                       3
Pre-Calculus                  1
Probability and Statistics    2
Computer Science              3
Portfolio Art                 1
Portfolio Photo               2
missing                       3
Economics                     1
Asia Studies                  2
Global Research               3
Chemistry                     1
Animal Bio                    2
Physics                       3
HS Health                     1
HS PE                         2
MS/HS PE                      3
Name: 3, dtype: int64

In [84]:
result.to_csv("processed_preference_data.csv",index_label=None)